In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
from transformers import DistilBertTokenizerFast,TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import tensorflow as tf
import tf_keras
import pandas as pd

In [3]:
df = pd.read_csv(r"/content/drive/MyDrive/preprocessed_dataset.csv")

In [4]:
df['label_encoded'] = df['label'].map({'Negative': 0, 'Positive': 1})
df

,label,title,content,clean_sentence,label_encoded
0,Negative,Expensive Junk,This product consists of a piece of thin flexi...,product consists piece thin flexible insulatin...,0
1,Negative,Toast too dark,"Even on the lowest setting, the toast is too d...",even lowest setting toast dark liking also lig...,0
2,Positive,Excellent imagery...dumbed down story,I enjoyed this disc. The video is stunning. I ...,enjoyed disc video stunning agree others story...,1
3,Negative,Are we pretending everyone is married?,The authors pretend that parents neither die n...,author pretend parent neither die divorce insi...,0
4,Negative,Not worth your time,"Might as well just use a knife, this product h...",might well use knife product hold next nothing...,0
...,...,...,...,...,...
19995,Negative,"Great sound, poor construction","I used these headphones at a desk job, listeni...",used headphone desk job listening ipod npr str...,0
19996,Negative,"""Stingy and egotistical gambler---not a role m...",Egotism is partly enthusiasm -- but mostly ign...,egotism partly enthusiasm mostly ignorance jor...,0
19997,Positive,RED!,I do love the color. Much more vibrant than in...,love color much vibrant picture lot room like ...,1
19998,Positive,A great silent.,Francesca Bertini was the first diva of Italia...,francesca bertini first diva italian movie por...,1


In [5]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['clean_sentence'].tolist(), df['label_encoded'].tolist(), test_size=0.2, random_state=42
)

In [6]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_enc = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_enc = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_enc), train_labels)).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_enc), test_labels)).batch(16)

In [11]:
model = TFDistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2,
    from_pt=True
    )
print('model_run')

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'cla

model_run


In [ ]:
model.compile(optimizer=tf_keras.optimizers.Adam(learning_rate=2e-5),loss=SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(train_dataset, validation_data=test_dataset, epochs=20)

In [13]:
model.compile(optimizer=tf_keras.optimizers.Adam(learning_rate=2e-7),loss=SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(train_dataset, validation_data=test_dataset, epochs=40,inital_epoch=20)

In [ ]:
model.save_pretrained("sentiment_model")
tokenizer.save_pretrained("sentiment_model")
print("here is the model")